In [1]:
import os
import pandas as pd

## Adding Logs

1. Get a views method identifier
2. Check line where the return value is Ok() or StatusCode()
4. Write in the error and success code
5. closer the file when done and move to the next

In [10]:
def success_log (indentation):

    return f"""
{indentation}// - start auto add
{indentation}logger.LogInfo("Message goes in here ");
{indentation}// - end auto add  
"""[1:]

def error_log (indentation):

    return f"""
{indentation}// - start auto add
{indentation}logger.Error("Message goes in here ");
{indentation}// - end auto add  
    """[1:]

In [12]:
files = os.listdir("./Controllers")

for file in files:
    file_path = "./Controllers/" + file
    file_contents  = []
    with open(file_path, "r") as f :
        file_contents = f.readlines()
        new_file_contents = [*file_contents]
        

        route_name = []

        added_lines = 0
        for line_no, line in enumerate(file_contents):
            if ("return Ok(" in line and "//" not in line ) or "return StatusCode(StatusCodes.Status200Ok" in line:
                indentation = " " * (line.index(line.strip()) - 1)
                log = success_log(indentation, route_name[-1])
                new_file_contents = [*new_file_contents[:(line_no + added_lines)], log, *new_file_contents[(line_no + added_lines):]]
                added_lines += 1
                
            elif ("return StatusCode(" in line ) and ("//" not in line) and ("Status200OK" not in line) and ("Error" in line):
                indentation = " " * (line.index(line.strip()) - 1)
                log = error_log(indentation, route_name[-1])
                new_file_contents = [*new_file_contents[:(line_no + added_lines)], log, *new_file_contents[(line_no + added_lines):]]
                added_lines += 1

            elif "public async Task" in line:
                indentation = " " * (line.index(line.strip()) - 1)            
                loop = True
                prev_line_no = line_no
                while loop:
                    prev_line = file_contents[prev_line_no - 1].strip()
                    if ("[Route(\"" in  prev_line): 
                        loop = False
                        route = prev_line[len("[Route(\"" ):-3].split("/")[0]
                        route_name.append(route)
                    
                    prev_line_no -= 1
        
        f.close()

    with open(file_path, "w") as  f:
        f.write("".join(new_file_contents))
        f.close()


## Generate a csv of all endpoints

In [ ]:
dirs = os.listdir("./Controllers")
endpoint = []
controller = []

for dir in dirs:
    file_path = "./Controllers/" + dir

    with open(file_path, "r") as f :
        file_contents = f.readlines()
        for line_no, line in enumerate(file_contents):
            if "public async Task" in line:
                indentation = " " * (line.index(line.strip()) - 1)            
                loop = True
                prev_line_no = line_no
                while loop:
                    prev_line = file_contents[prev_line_no - 1].strip()
                    if ("[Route(\"" in  prev_line) and ("//" not in prev_line): 
                        loop = False
                        route = prev_line[len("[Route(\"" ):-3]
                        
                        endpoint.append(route)
                        controller.append(dir[:dir.index("Controller") ])
                    
                    prev_line_no -= 1
        
        f.close()

pd.DataFrame({"endpoints": endpoint, "controller": controller}).to_csv("contents.csv", index= False)


## Adding Roles
1. Locate the Authorize Decorator
2. Generate a new Decorator based on the CSV Role sepecification.
3. Overwrite the original Authorize decorator with the new one.

In [14]:

#         
roles = "HOD, HODHR, \tHRFIN"
def gen_authorize_line(roles):
    roles = [f"UserRoles.{role.strip()}" for role in roles.split(",")]
    return f"""\t\t[Authorize(Roles = {' + "," + '.join(roles)} ) ]\n"""

gen_authorize_line(roles)

'\t\t[Authorize(Roles = UserRoles.HOD + "," + UserRoles.HODHR + "," + UserRoles.HRFIN ) ]\n'

In [28]:
files = os.listdir("./Controllers")
df = pd.read_csv("contents.csv")

for file in files:
    authorize_decorator_line_no = []
    endpoint = []
    file_path = "./Controllers/" + file
    with open(file_path, "r") as f :
        file_contents = f.readlines()
        new_file_contents = [*file_contents]
        controller =[]
        for line_no, line in enumerate(file_contents):
            found_authorize = False

            if "public async Task" in line:
                indentation = " " * (line.index(line.strip()) - 1)  

                # Get the route
                loop = True
                prev_line_no = line_no 
                route = ""
                while loop:
                    prev_line = file_contents[prev_line_no - 1].strip()
                    if ("[Route(\"" in  prev_line) and ("//" not in prev_line): 
                        loop = False
                        route = prev_line[len("[Route(\"" ):-3]
                        
                        endpoint.append(route)
                        controller.append(file[:file.index("Controller") ])
                    elif(prev_line_no == 0):
                        loop = False
                    else:
                        prev_line_no -= 1
                        
                # get the authorize line number
                loop = True
                prev_line_no = line_no
                while loop:
                    prev_line = file_contents[prev_line_no - 1].strip()
                    if ("[Authorize" in  prev_line) and ("//" not in prev_line): 
                        authorize_decorator_line_no.append(prev_line_no)
                        loop = False
                    elif(prev_line_no == 0):
                        loop = False
                    else:
                        prev_line_no -= 1
                        pass
                    found_authorize = True
                
                # replace authorize line
                get_endpoint = endpoint[-1]
                get_controller = file[:-3].split("Controller")[0]
                roles_query = df.query(f'endpoints == "{get_endpoint}" and controller == "{get_controller}"')["roles"].to_list()[0]
                
                if(roles_query and found_authorize):
                    print(prev_line_no)
                    new_file_contents[prev_line_no -1] = gen_authorize_line(roles_query)
                    found_authorize = False
                if(roles_query):
                    new_file_contents = [*new_file_contents[:line_no], gen_authorize_line(roles_query), new_file_contents[line_no:]]
        f.close()
    with open(file_path, "w") as f:
        f.write("".join(new_file_contents))
        f.close()
        print(file, ": DONE!")

#         [Authorize(Roles = UserRoles.HOD + "," + UserRoles.HODADMIN + "," + UserRoles.HODHR + "," + UserRoles.HODIT + "," + UserRoles.HODFIN) ]

0
0
0
0
0
0
590


IndexError: list assignment index out of range

In [ ]:

start_path = "."
for root, dirs, files in os.walk(start_path):
    for file in files:
        
        if file.split(".")[-1] == "js":
            print(f"{root}\\{file}")
            file_path = f"{root}\\{file}"
            # read file contents
            
            contents = ""
            with open(f"{root}\\{file}", "r") as f:
                contents = f.read()
                f.close()
            
            contents = contents.replace("localStorage.", "secureLocalStorage.")
            contents = f'''import  secureLocalStorage  from  "react-secure-storage";\n{contents} '''
            
            with open(f"{root}\\{file}", "w") as f:
                f.write(contents)


                f.close()
                
                print("updated")